# 0. Antecedentes

El Data Set contiene datos de dos hoteles diferentes. Un hotel Resort y un hotel de ciudad.

De la publicación: https://www.sciencedirect.com/science/article/pii/S2352340918315191 sabemos que ambos hoteles están ubicados en Portugal (sur de Europa) ("H1 en la región turística de Algarve y H2 en la ciudad de Lisboa"). La distancia entre estos dos lugares es ca. 280 km en coche y ambas localidades limitan con el atlántico norte.

Los datos contienen "reservas que deben llegar entre el 1 de julio de 2015 y el 31 de agosto de 2017".

Topics covered and questions to answer from the data:



*   ¿De dónde vienen los invitados? 
*   ¿Cuánto pagan los huéspedes por una habitación por noche? 
*   ¿Cómo varía el precio por noche a lo largo del año? 
*   ¿Cuáles son los meses más ocupados? 
*   ¿Cuánto tiempo se queda la gente en los hoteles?
*   Reservas por segmento de mercado 
*   ¿Cuántas reservas se cancelaron? 
*   ¿Qué mes tiene el mayor número de cancelaciones?



# 1. Importacion de Paquetes y Cargado de Datos

**Importación de paquetes**

In [ ]:
# Tratamiento y visualizacion de datos:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import folium

# Predicciones ML:
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

**Configuración y Cargando Datos**

In [ ]:
# Establecer algunas opciones de visualización:
sns.set(style="whitegrid")
pd.set_option("display.max_columns", 36)
pd.set_option('precision', 2)

# Cargando datos:
file_path = "/content/drive/MyDrive/Data Sets/Hotel DataSet/hotel_bookings.csv"
hotel_data = pd.read_csv(file_path)

# 2. EDA

In [ ]:
hotel_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [ ]:
# Revisando missing values
hotel_data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [ ]:
hotel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

# 3. Transformacion

In [ ]:
# Reemplazar valores perdidos:nan
# agent: Si no se proporciona una agencia, lo más probable es 
#        que la reserva se haya realizado sin ayuda de una.
# company: Si no se da ninguno, lo más probable es que sea privado.
nan_replacements = {"children:": 0,"country": "Unknown", "agent": 0, "company": 0}
hotel_data_cln = hotel_data.fillna(nan_replacements)

# "meal" contiene valores "Indefinido", que es igual a SC.
hotel_data_cln["meal"].replace("Undefined", "SC", inplace=True)

hotel_data_cln["children"].replace(np.nan,0, inplace=True)

# Algunas filas contienen entradas con 0 adultos, 0 niños y 0 bebés. 
# Estoy eliminando estas entradas sin invitados.
zero_guests = list(hotel_data_cln.loc[hotel_data_cln["adults"]
                   + hotel_data_cln["children"]
                   + hotel_data_cln["babies"]==0].index)
hotel_data_cln.drop(hotel_data_cln.index[zero_guests], inplace=True)

In [ ]:
# ¿Cuántos datos quedan?
hotel_data_cln.shape

(119210, 32)

In [ ]:
hotel_data_cln.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

**Segmentacion**

In [ ]:
column_list = ['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date']
for i in column_list:
  print(i, ":", hotel_data_cln[i].unique())

hotel : ['Resort Hotel' 'City Hotel']
is_canceled : [0 1]
lead_time : [342 737   7  13  14   0   9  85  75  23  35  68  18  37  12  72 127  78
  48  60  77  99 118  95  96  69  45  40  15  36  43  70  16 107  47 113
  90  50  93  76   3   1  10   5  17  51  71  63  62 101   2  81 368 364
 324  79  21 109 102   4  98  92  26  73 115  86  52  29  30  33  32   8
 100  44  80  97  64  39  34  27  82  94 110 111  84  66 104  28 258 112
  65  67  55  88  54 292  83 105 280 394  24 103 366 249  22  91  11 108
 106  31  87  41 304 117  59  53  58 116  42 321  38  56  49 317   6  57
  19  25 315 123  46  89  61 312 299 130  74 298 119  20 286 136 129 124
 327 131 460 140 114 139 122 137 126 120 128 135 150 143 151 132 125 157
 147 138 156 164 346 159 160 161 333 381 149 154 297 163 314 155 323 340
 356 142 328 144 336 248 302 175 344 382 146 170 166 338 167 310 148 165
 172 171 145 121 178 305 173 152 354 347 158 185 349 183 352 177 200 192
 361 207 174 330 134 350 334 283 153 197 133 241 193 2

In [ ]:
unique_dic = {}
for i in column_list:
  unique_dic[i] = len(hotel_data_cln[i].unique())
unique_dic

{'adr': 8866,
 'adults': 14,
 'agent': 334,
 'arrival_date_day_of_month': 31,
 'arrival_date_month': 12,
 'arrival_date_week_number': 53,
 'arrival_date_year': 3,
 'assigned_room_type': 11,
 'babies': 5,
 'booking_changes': 19,
 'children': 5,
 'company': 349,
 'country': 178,
 'customer_type': 4,
 'days_in_waiting_list': 127,
 'deposit_type': 3,
 'distribution_channel': 5,
 'hotel': 2,
 'is_canceled': 2,
 'is_repeated_guest': 2,
 'lead_time': 479,
 'market_segment': 8,
 'meal': 4,
 'previous_bookings_not_canceled': 73,
 'previous_cancellations': 15,
 'required_car_parking_spaces': 5,
 'reservation_status': 3,
 'reservation_status_date': 926,
 'reserved_room_type': 9,
 'stays_in_week_nights': 33,
 'stays_in_weekend_nights': 17,
 'total_of_special_requests': 6}

In [ ]:
hotel_data_cln['adr']

0           0.00
1           0.00
2          75.00
3          75.00
4          98.00
           ...  
119385     96.14
119386    225.43
119387    157.71
119388    104.40
119389    151.20
Name: adr, Length: 119210, dtype: float64

In [ ]:
# 'Resort Hotel' 'City Hotel'
Resort_Hotel = hotel_data_cln[hotel_data_cln['hotel'] == 'Resort Hotel']
City_Hotel = hotel_data_cln[hotel_data_cln['hotel'] == 'City Hotel']

In [ ]:
Resort_Hotel.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00,40047.00
mean,0.28,92.69,2016.12,27.14,15.82,1.19,3.13,1.87,0.13,0.01,0.04,0.10,0.15,0.29,173.02,18.69,0.52,94.98,0.14,0.62
std,0.45,97.29,0.72,14.00,8.88,1.15,2.46,0.70,0.45,0.12,0.21,1.34,1.00,0.72,117.91,73.38,7.38,61.43,0.35,0.81
min,0.00,0.00,2015.00,1.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6.38,0.00,0.00
25%,0.00,10.00,2016.00,16.00,8.00,0.00,1.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,36.00,0.00,0.00,50.00,0.00,0.00
50%,0.00,57.00,2016.00,28.00,16.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,240.00,0.00,0.00,75.00,0.00,0.00
75%,1.00,155.00,2017.00,38.00,24.00,2.00,5.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,241.00,0.00,0.00,125.00,0.00,1.00
max,1.00,737.00,2017.00,53.00,31.00,19.00,50.00,55.00,10.00,2.00,1.00,26.00,30.00,17.00,535.00,543.00,185.00,508.00,8.00,5.00


In [ ]:
City_Hotel.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,7.92e+04,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00,79163.00
mean,0.42,109.88,2016.17,27.18,15.79,0.79,2.18,1.85,0.09,4.95e-03,0.02,0.08,0.13,0.18,25.25,6.71,3.23,105.50,0.02,0.55
std,0.49,110.96,0.70,13.39,8.73,0.88,1.43,0.50,0.37,8.44e-02,0.16,0.42,1.69,0.59,54.09,39.89,20.89,43.41,0.15,0.78
min,0.00,0.00,2015.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,23.00,2016.00,17.00,8.00,0.00,1.00,2.00,0.00,0.00e+00,0.00,0.00,0.00,0.00,7.00,0.00,0.00,79.20,0.00,0.00
50%,0.00,74.00,2016.00,27.00,16.00,1.00,2.00,2.00,0.00,0.00e+00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,99.96,0.00,0.00
75%,1.00,164.00,2017.00,38.00,23.00,2.00,3.00,2.00,0.00,0.00e+00,0.00,0.00,0.00,0.00,14.00,0.00,0.00,126.00,0.00,1.00
max,1.00,629.00,2017.00,53.00,31.00,14.00,34.00,4.00,3.00,1.00e+01,1.00,21.00,72.00,18.00,509.00,497.00,391.00,5400.00,3.00,5.00


# 4. Exportacion


**Exportamos los df por separado de Hoteles Resort y Ciudad**
```
Resort_Hotel_path = "/content/drive/MyDrive/Data Sets/Hotel DataSet/Resort_Hotel.csv"
City_Hotel_path = "/content/drive/MyDrive/Data Sets/Hotel DataSet/City_Hotel.csv"
Resort_Hotel.to_csv(Resort_Hotel_path)
City_Hotel.to_csv(City_Hotel_path)
```

**Exportamos el df limpio combinado de Hoteles Resort y Ciudad**



```
hotel_data_cln_path = "/content/drive/MyDrive/Data Sets/Hotel DataSet/hotel_data_cln.csv"
hotel_data_cln.to_csv(hotel_data_cln_path)
```



